In [422]:
from tkinter import YView, XView

import numpy as np
import pandas as pd
np.random.seed = 123
data = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

In [423]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [424]:
data['Age'].fillna(data['Age'].mean(),inplace=True)
data['Cabin'].fillna('N',inplace=True)
data['Embarked'].fillna('N',inplace=True)

/var/folders/9d/lcchyqrx6jq_hz6pm4d1j7z80000gn/T/ipykernel_58162/3057721139.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Age'].fillna(data['Age'].mean(),inplace=True)
/var/folders/9d/lcchyqrx6jq_hz6pm4d1j7z80000gn/T/ipykernel_58162/3057721139.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values a

In [425]:
data.loc[data["Sex"] == "male", "Sex_encode"] = 0
data.loc[data["Sex"] == "female", "Sex_encode"] = 1

In [426]:
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 피처 제거
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩 수행.
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [427]:
Y = data['Survived']
X = data.drop('Survived', axis=1)


In [428]:
X = transform_features(X)

/var/folders/9d/lcchyqrx6jq_hz6pm4d1j7z80000gn/T/ipykernel_58162/3807705002.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].mean(), inplace=True)
/var/folders/9d/lcchyqrx6jq_hz6pm4d1j7z80000gn/T/ipykernel_58162/3807705002.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

In [429]:
import numpy as np

# 로그 변환 적용 (값이 0 이하인 경우를 방지하기 위해 작은 값 더하기)
X['Fare'] = np.log(X['Fare'] + 1)
X['Age'] = np.log(X['Age'] + 1)



In [430]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, Y, \
                                                  test_size=0.2, random_state=11)

In [431]:
def batch_sampler(data, target, batch_size):
    tot = len(data)
    total_iter = tot // batch_size
    for i in range(total_iter):
        yield (data[i * batch_size:(i + 1) * batch_size],
               target[i * batch_size:(i + 1) * batch_size])


sampler = batch_sampler(data=X_train, target=y_train, batch_size=32)
L = len(X_train) // 32
for i in range(L):
    next(sampler)
print(X_train)

     Pclass  Sex       Age  SibSp  Parch      Fare  Cabin  Embarked  \
333       3    1  2.833213      2      0  2.944439      7         3   
662       1    1  3.871201      0      0  3.280441      4         3   
382       3    1  3.496508      0      0  2.188856      7         3   
331       1    1  3.839452      0      0  3.384390      2         3   
149       2    1  3.761200      0      0  2.639057      7         3   
..      ...  ...       ...    ...    ...       ...    ...       ...   
269       1    0  3.583519      0      0  4.917301      2         3   
337       1    0  3.737670      0      0  4.908972      4         0   
91        3    1  3.044522      0      0  2.180892      7         3   
80        3    1  3.135494      0      0  2.302585      7         3   
703       3    1  3.258097      0      0  2.168105      7         2   

     Sex_encode  
333         0.0  
662         0.0  
382         0.0  
331         0.0  
149         0.0  
..          ...  
269         1.0  
337

In [432]:
import numpy as np
from tqdm import tqdm
class LogisticRegression:
    def __init__(self, learning_rate=0.01, epochs=400, n_features=9):
        # 초기 가중치를 무작위로 설정
        self.weights = np.random.randn(n_features)
        self.b = np.random.randn()
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.loss_history = []

    def sigmoid(self, z):
        """Sigmoid 활성화 함수"""
        return 1.0 / (1 + np.exp(-z))

    def compute_loss(self, preds, target):
        """교차 엔트로피 손실 계산"""
        # 로그의 안정성을 위해 값들을 클리핑
        preds = np.clip(preds, 1e-9, 1 - 1e-9)  # 0과 1의 범위를 벗어나지 않도록 함
        loss = -((target * np.log(preds)) + (1 - target) * np.log(1 - preds)).mean()
        return loss


    def fit(self, X, y, batch_sampler):
        """모델 훈련 함수"""
        for epoch in tqdm(range(self.epochs)):
            sampler = batch_sampler(data=X, target=y, batch_size=32)
            batch_loss = []
    
            for X_batch, Y_batch in sampler:
                # 순방향 전파
                preds = self.sigmoid(np.dot(X_batch, self.weights) + self.b)
    
                # 손실 계산
                loss = self.compute_loss(preds, Y_batch)
                batch_loss.append(loss)
    
                # 그래디언트 계산
                dloss_dz = preds - Y_batch
                weights_deriv = np.dot(X_batch.T, dloss_dz) / X_batch.shape[0]
                b_deriv = dloss_dz.mean()
    
                # 가중치 및 바이어스 업데이트
                self.weights -= self.learning_rate * weights_deriv
                self.b -= self.learning_rate * b_deriv
    
            # Epoch별 손실 저장
            epoch_loss = np.mean(batch_loss)
            self.loss_history.append(epoch_loss)
            
            # 에포크마다 손실 출력
            print(f"Epoch {epoch + 1}/{self.epochs} - Loss: {epoch_loss:.4f}")


    def predict(self, X):
        """모델 예측 함수"""
        linear_comb = np.dot(X, self.weights) + self.b
        return self.sigmoid(linear_comb) > 0.55

    def accuracy(self, X, y):
        """정확도 계산 함수"""
        preds = self.predict(X)
        return (preds == y).mean() * 100




In [433]:
model = LogisticRegression(learning_rate=0.005, epochs=200, n_features=9)
model.fit(X_train, y_train, batch_sampler=batch_sampler)

 15%|█▌        | 30/200 [00:00<00:01, 146.29it/s]

Epoch 1/200 - Loss: 1.7556
Epoch 2/200 - Loss: 0.8783
Epoch 3/200 - Loss: 0.7995
Epoch 4/200 - Loss: 0.7833
Epoch 5/200 - Loss: 0.7721
Epoch 6/200 - Loss: 0.7616
Epoch 7/200 - Loss: 0.7516
Epoch 8/200 - Loss: 0.7418
Epoch 9/200 - Loss: 0.7325
Epoch 10/200 - Loss: 0.7235
Epoch 11/200 - Loss: 0.7148
Epoch 12/200 - Loss: 0.7065
Epoch 13/200 - Loss: 0.6986
Epoch 14/200 - Loss: 0.6909
Epoch 15/200 - Loss: 0.6836
Epoch 16/200 - Loss: 0.6766
Epoch 17/200 - Loss: 0.6698
Epoch 18/200 - Loss: 0.6634
Epoch 19/200 - Loss: 0.6572
Epoch 20/200 - Loss: 0.6513
Epoch 21/200 - Loss: 0.6456
Epoch 22/200 - Loss: 0.6401
Epoch 23/200 - Loss: 0.6349
Epoch 24/200 - Loss: 0.6299
Epoch 25/200 - Loss: 0.6251
Epoch 26/200 - Loss: 0.6205
Epoch 27/200 - Loss: 0.6160
Epoch 28/200 - Loss: 0.6118
Epoch 29/200 - Loss: 0.6077
Epoch 30/200 - Loss: 0.6038
Epoch 31/200 - Loss: 0.6000
Epoch 32/200 - Loss: 0.5963
Epoch 33/200 - Loss: 0.5928
Epoch 34/200 - Loss: 0.5895
Epoch 35/200 - Loss: 0.5862
Epoch 36/200 - Loss: 0.5831
E

 30%|███       | 60/200 [00:00<00:00, 145.45it/s]

Epoch 46/200 - Loss: 0.5570
Epoch 47/200 - Loss: 0.5549
Epoch 48/200 - Loss: 0.5528
Epoch 49/200 - Loss: 0.5508
Epoch 50/200 - Loss: 0.5488
Epoch 51/200 - Loss: 0.5469
Epoch 52/200 - Loss: 0.5451
Epoch 53/200 - Loss: 0.5433
Epoch 54/200 - Loss: 0.5415
Epoch 55/200 - Loss: 0.5398
Epoch 56/200 - Loss: 0.5382
Epoch 57/200 - Loss: 0.5366
Epoch 58/200 - Loss: 0.5351
Epoch 59/200 - Loss: 0.5336
Epoch 60/200 - Loss: 0.5321
Epoch 61/200 - Loss: 0.5307
Epoch 62/200 - Loss: 0.5293
Epoch 63/200 - Loss: 0.5280
Epoch 64/200 - Loss: 0.5267
Epoch 65/200 - Loss: 0.5254
Epoch 66/200 - Loss: 0.5242
Epoch 67/200 - Loss: 0.5230
Epoch 68/200 - Loss: 0.5218
Epoch 69/200 - Loss: 0.5207
Epoch 70/200 - Loss: 0.5196
Epoch 71/200 - Loss: 0.5185
Epoch 72/200 - Loss: 0.5175
Epoch 73/200 - Loss: 0.5164
Epoch 74/200 - Loss: 0.5154
Epoch 75/200 - Loss: 0.5144


 45%|████▌     | 90/200 [00:00<00:00, 145.56it/s]

...

Epoch 165/200 - Loss: 0.4734
Epoch 166/200 - Loss: 0.4732
Epoch 167/200 - Loss: 0.4730
Epoch 168/200 - Loss: 0.4728
Epoch 169/200 - Loss: 0.4726
Epoch 170/200 - Loss: 0.4724
Epoch 171/200 - Loss: 0.4722
Epoch 172/200 - Loss: 0.4720
Epoch 173/200 - Loss: 0.4718
Epoch 174/200 - Loss: 0.4716
Epoch 175/200 - Loss: 0.4714
Epoch 176/200 - Loss: 0.4713
Epoch 177/200 - Loss: 0.4711
Epoch 178/200 - Loss: 0.4709
Epoch 179/200 - Loss: 0.4707
Epoch 180/200 - Loss: 0.4706
Epoch 181/200 - Loss: 0.4704


 98%|█████████▊| 196/200 [00:01<00:00, 144.98it/s]

Epoch 182/200 - Loss: 0.4702
Epoch 183/200 - Loss: 0.4701
Epoch 184/200 - Loss: 0.4699
Epoch 185/200 - Loss: 0.4698
Epoch 186/200 - Loss: 0.4696
Epoch 187/200 - Loss: 0.4694
Epoch 188/200 - Loss: 0.4693
Epoch 189/200 - Loss: 0.4691
Epoch 190/200 - Loss: 0.4690
Epoch 191/200 - Loss: 0.4688
Epoch 192/200 - Loss: 0.4687
Epoch 193/200 - Loss: 0.4685
Epoch 194/200 - Loss: 0.4684
Epoch 195/200 - Loss: 0.4683
Epoch 196/200 - Loss: 0.4681


100%|██████████| 200/200 [00:01<00:00, 145.02it/s]

Epoch 197/200 - Loss: 0.4680
Epoch 198/200 - Loss: 0.4679
Epoch 199/200 - Loss: 0.4677
Epoch 200/200 - Loss: 0.4676


In [434]:
# X_test에 대한 예측값 생성
predictions = model.predict(X_test)

# 정확도 계산
def calculate_accuracy(predictions, y_test):
    """예측값과 실제값을 비교하여 정확도를 계산"""
    correct_predictions = (predictions == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100
    return accuracy

# y_test와 비교하여 정확도 계산
accuracy = calculate_accuracy(predictions, y_test)
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 86.03%


In [435]:
model.accuracy(X_test, y_test)

np.float64(86.03351955307262)